<a href="https://colab.research.google.com/github/suriya43426/Classification_CNN_Crystallization/blob/master/3_%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%AA%E0%B8%A3%E0%B9%89%E0%B8%B2%E0%B8%87_Graph_%E0%B8%94%E0%B9%89%E0%B8%A7%E0%B8%A2_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Helper-function สำหรับการ Flatten เลเยอร์

Convolution สร้างเทนเซอร์เอาต์พุตที่มี 4 มิติ เราจะเพิ่มเลเยอร์ที่ Fully Connectted หลังจากเลเยอร์คอนโวลูชั่นดังนั้นเราจำเป็นต้องลดเทนเซอร์ 4 สลัวเป็น 2 สลัวซึ่งสามารถใช้เป็นอินพุตไปยังเลเยอร์ที่เชื่อมต่อแบบเต็มได้

In [ ]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

### Helper-function สำหรับการสร้าง Fully-Connected เลเยอร์

ฟังก์ชันนี้จะสร้างเลเยอร์ใหม่ที่เชื่อมต่ออย่างสมบูรณ์ในกราฟการคำนวณสำหรับ TensorFlow ไม่มีอะไรคำนวณได้ที่นี่เราแค่เพิ่มสูตรทางคณิตศาสตร์ลงในกราฟ TensorFlow

โดยจะสันนิษฐานว่าอินพุตเป็นแบบเทนเซอร์ 2 มิติ`[num_images, num_inputs]`. เอาต์พุตเป็นแบบเทนเซอร์แบบ 2 มิติ `[num_images, num_outputs]`.

In [ ]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

### Placeholder variables

ตัวแปรตัวยึดทำหน้าที่เป็นอินพุตของกราฟการคำนวณ TensorFlow ที่เราอาจเปลี่ยนแปลงทุกครั้งที่เราเรียกใช้กราฟ เราเรียกสิ่งนี้ว่าการให้อาหารตัวแปรตัวยึดตำแหน่งและแสดงเพิ่มเติมด้านล่าง

อันดับแรกเรากำหนดตัวแปรตัวยึดตำแหน่งสำหรับภาพที่ป้อน สิ่งนี้ช่วยให้เราสามารถเปลี่ยนภาพที่ป้อนลงในกราฟ TensorFlow นี่คือสิ่งที่เรียกว่าเทนเซอร์ซึ่งหมายความว่ามันเป็นเวกเตอร์หรือเมทริกซ์หลายมิติ ชนิดข้อมูลถูกตั้งค่าเป็น `float32` และรูปร่างถูกตั้งค่าเป็น `[None, img_size_flat]`โดยที่`None` หมายความว่า Tensor อาจเก็บภาพได้ตามจำนวนที่กำหนดโดยแต่ละภาพเป็นเวกเตอร์ที่มีความยาว `img_size_flat`.

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

Convolution Layer คาดว่า x จะถูกเข้ารหัสเป็น Tensor 4 มิติดังนั้นเราจึงต้องจัดรูปแบบใหม่เพื่อให้รูปร่างของ instead`[num_images, img_height, img_width, num_channels]`. แทน โปรดทราบว่า `img_height == img_width == img_size` และ `num_images`สามารถอนุมานได้โดยอัตโนมัติโดยใช้ -1 สำหรับขนาดของมิติแรก ดังนั้นการดำเนินการก่อร่างใหม่คือ:


In [ ]:
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])

ต่อไปเรามีตัวแปร placeholder variable สำหรับ labels แบบจริง ซึ่งเกี่ยวข้องกับภาพที่ป้อนในตัวแปรตัวยึดตำแหน่ง `x` รูปร่างของตัวแปรตัวยึดตำแหน่งนี้คือ `[None, num_classes]` ซึ่งหมายความว่าอาจมีจำนวน labels โดยพลการและแต่ละ labels เป็นเวกเตอร์ของความยาว`num_classes` ซึ่งเป็น 10 ในกรณีนี้

In [ ]:
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

เราสามารถมีตัวแปรตัว placeholder สำหรับหมายเลข class ได้ แต่เราจะคำนวณโดยใช้ argmax แทน โปรดทราบว่านี่เป็นตัวดำเนินการ TensorFlow ดังนั้นจึงไม่มีการคำนวณอะไรในจุดนี้

In [ ]:
y_true_cls = tf.argmax(y_true, axis=1)